In [1]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as img
from PIL import Image
import copy
import cv2
import os

### Inputing the Dataset

REPLAY ATTACK

In [2]:
def load_images_from_folder(folder,l):

    images = []
    labels = []
    size = 64
    for label in os.listdir(folder):
        
        label_path = os.path.join(folder, label)
        if os.path.isdir(label_path):

            for filename in os.listdir(label_path):

                img_path = os.path.join(label_path, filename)
                if img_path.endswith(('.jpg', '.jpeg', '.png', '.bmp')):
                    
                    img = cv2.resize(cv2.imread(img_path),(size,size))
                    if img is not None:

                        images.append(img)
                        labels.append(l)
                    

    return images, labels

# Train Dataset real
path = r"F:\IVP_Project\Dataset\Dataset\replay_attack_cropped\Dataset\Replay Attack\Dataset\train\real"
replay_img_real_train, replay_label_r_train = load_images_from_folder(path, 1)

# Train Dataset attack

path = r"F:\IVP_Project\Dataset\Dataset\replay_attack_cropped\Dataset\Replay Attack\Dataset\train\attack\hand"
replay_img_spoof_train_hand, replay_label_s_train_hand = load_images_from_folder(path, -1)


path = r"F:\IVP_Project\Dataset\Dataset\replay_attack_cropped\Dataset\Replay Attack\Dataset\train\attack\fixed"
replay_img_spoof_train_fixed, replay_label_s_train_fixed = load_images_from_folder(path, -1)

replay_img_spoof_train = replay_img_spoof_train_fixed + replay_img_spoof_train_hand 
replay_label_s_train = replay_label_s_train_fixed + replay_label_s_train_hand

replay_img_train = np.array(replay_img_real_train + replay_img_spoof_train)
replay_label_train = np.array(replay_label_r_train + replay_label_s_train)



# test Dataset real
path = r"F:\IVP_Project\Dataset\Dataset\replay_attack_cropped\Dataset\Replay Attack\Dataset\test\real"
replay_img_real_test, replay_label_r_test = load_images_from_folder(path, 1)

# test Dataset attack

path = r"F:\IVP_Project\Dataset\Dataset\replay_attack_cropped\Dataset\Replay Attack\Dataset\test\attack\hand"
replay_img_spoof_test_hand, replay_label_s_test_hand = load_images_from_folder(path, -1)


path = r"F:\IVP_Project\Dataset\Dataset\replay_attack_cropped\Dataset\Replay Attack\Dataset\test\attack\fixed"
replay_img_spoof_test_fixed, replay_label_s_test_fixed = load_images_from_folder(path, -1)

replay_img_spoof_test = replay_img_spoof_test_fixed + replay_img_spoof_test_hand 
replay_label_s_test = replay_label_s_test_fixed + replay_label_s_test_hand

replay_img_test = np.array(replay_img_real_test + replay_img_spoof_test)
replay_label_test = np.array(replay_label_r_test + replay_label_s_test)

LCC

In [3]:
def load_images_from_folder_modified(folder,l):

    images = []
    labels = []
    size = 64
    for label in os.listdir(folder):
        #for filename in os.listdir(label):

        img_path = os.path.join(folder, label) 
        #os.path.join(label, filename)
        
        if img_path.endswith(('.jpg', '.jpeg', '.png', '.bmp')):
            img = cv2.resize(cv2.imread(img_path),(size,size))
            if img is not None:

                images.append(img)
                labels.append(l)
                    

    return images, labels

# Train Dataset real
path = r"F:\IVP_Project\Dataset\Dataset\lcc_fasd_cropped\LCC_FASD\LCC_FASD_training\real"
LCC_img_real_train, LCC_label_r_train = load_images_from_folder_modified(path, 1)

# Train Dataset attack

path = r"F:\IVP_Project\Dataset\Dataset\lcc_fasd_cropped\LCC_FASD\LCC_FASD_training\spoof"
LCC_img_spoof_train, LCC_label_s_train = load_images_from_folder_modified(path, -1)

LCC_img_train = np.array(LCC_img_real_train + LCC_img_spoof_train)
LCC_label_train = np.array(LCC_label_r_train + LCC_label_s_train)


# test Dataset real
path = r"F:\IVP_Project\Dataset\Dataset\lcc_fasd_cropped\LCC_FASD\LCC_FASD_evaluation\real"
LCC_img_real_test, LCC_label_r_test = load_images_from_folder_modified(path, 1)

# test Dataset attack

path = r"F:\IVP_Project\Dataset\Dataset\lcc_fasd_cropped\LCC_FASD\LCC_FASD_evaluation\spoof"
LCC_img_spoof_test, LCC_label_s_test = load_images_from_folder_modified(path, -1)

LCC_img_test = np.array(LCC_img_real_test + LCC_img_spoof_test)
LCC_label_test = np.array(LCC_label_r_test + LCC_label_s_test)

NUAA

In [4]:
def load_images_from_folder(folder,l):

    images = []
    labels = []
    size = 64

    for label in os.listdir(folder):
        
        label_path = os.path.join(folder, label)
        if os.path.isdir(label_path):

            for filename in os.listdir(label_path):

                img_path = os.path.join(label_path, filename)
                if img_path.endswith(('.jpg', '.jpeg', '.png', '.bmp')):

                    img = cv2.resize(cv2.imread(img_path),(size,size))
                    if img is not None:

                        images.append(img)
                        labels.append(l)

    return images, labels

# Original Faces(Client) with label 1
path = r"F:\IVP_Project\Dataset\Dataset\nuaa_cropped\NUAA\Detectedface\ClientFace"

img_real, label_r = load_images_from_folder(path, 1)

# Spoof Faces with label -1
path = r"F:\IVP_Project\Dataset\Dataset\nuaa_cropped\NUAA\Detectedface\ImposterFace"
img_spoof, label_s = load_images_from_folder(path, -1)


### Defining the LBP

In [5]:
from skimage import feature
import numpy as np

class LocalBinaryPatterns:
    def __init__(self, numPoints, radius):
        # store the number of points and radius
        self.numPoints = numPoints
        self.radius = radius
        
    def lbp_calculate(self, image):
        # compute the Local Binary Pattern representation
        # of the image, and then use the LBP representation
        # to build the histogram of patterns
        lbp_image = feature.local_binary_pattern(image, self.numPoints, self.radius, method="uniform")
        
        lbp_r = np.asarray(np.round(lbp_image)).astype(np.uint8)
        return lbp_r
	

def plot_histogram(image, if_plot=1):
    # Calculate histogram
    hist = cv2.calcHist([image], [0], None, [256], [0, 256])

    # Plot histogram
    if if_plot==1:
        plt.plot(hist)
        plt.title('Histogram')
        plt.xlabel('Pixel Intensity')
        plt.ylabel('Frequency')
        plt.show()
        
    return hist


### Feature Vector Generation

In [6]:
def region(image,n=3):

    x = 24

    y = 4

    #overlapping region  = 24*4 = 96

    regions = []

    regions.append(image[0:x,0:x])

    regions.append(image[x-y:2*x-y,0:x])

    regions.append(image[2*x-2*y:,0:x])

    regions.append(image[0:x,x-y:2*x-y])

    regions.append(image[x-y:2*x-y,x-y:2*x-y])

    regions.append(image[2*x-2*y:,x-y:2*x-y])

    regions.append(image[0:x,2*x-2*y:])

    regions.append(image[x-y:2*x-y,2*x-2*y:])

    regions.append(image[2*x-2*y:,2*x-2*y:])


    regions = np.array(regions)

    return regions


def feature_LBP(img1):
    size = 64

    img  = cv2.cvtColor(cv2.resize(img1,(size,size)), cv2.COLOR_BGR2GRAY) 
        
    LBP_1 = LocalBinaryPatterns(8,1)
    LBP_2 = LocalBinaryPatterns(8,2)
    LBP_3 = LocalBinaryPatterns(16,2)

    lbp_img_81 = LBP_1.lbp_calculate(img)
   
    regions = region(lbp_img_81)

    features = []

    for r in regions:

        feature_histogram = plot_histogram(r,0)

        features.append(feature_histogram[:10])

# Adding the full image image properties
        
    lbp_img_82 = LBP_2.lbp_calculate(img)
    feature_histogram = plot_histogram(lbp_img_82,0)
    features.append(feature_histogram[:10])
    
    lbp_img_162 = LBP_3.lbp_calculate(img)
    feature_histogram = plot_histogram(lbp_img_162,0)
    features.append(feature_histogram[:10])
    
    features = (np.array(features)).reshape(110)

    return features


REPLAY ATTACK

In [7]:
from tqdm import tqdm

# FOR REPLAY ATTACK DATASET
feature_vec_replay_train = []
feature_vec_replay_test = []

replay_train_images = replay_img_train
replay_test_images = replay_img_test

# Loop over images in image_real with tqdm progress bar
for img in tqdm(replay_train_images, desc="Processing replay dataset train images"):
    feature_vec_replay_train.append(feature_LBP(img))

# Loop over images in image_spoof with tqdm progress bar
for img in tqdm(replay_test_images, desc="Processing replay dataset test images"):
    feature_vec_replay_test.append(feature_LBP(img))

# Convert lists to numpy arrays
feature_vector_replay_train = np.array(feature_vec_replay_train)
feature_vector_replay_test = np.array(feature_vec_replay_test)


Processing replay dataset test images: 100%|██████████| 4800/4800 [00:11<00:00, 403.90it/s]


LCC

In [8]:
from tqdm import tqdm

# FOR LCC DATASET
feature_vec_LCC_train = []
feature_vec_LCC_test = []

LCC_train_images = LCC_img_train
LCC_test_images = LCC_img_test

# Loop over images in image_real with tqdm progress bar
for img in tqdm(LCC_train_images, desc="Processing LCC dataset train images"):
    feature_vec_LCC_train.append(feature_LBP(img))

# Loop over images in image_spoof with tqdm progress bar
for img in tqdm(LCC_test_images, desc="Processing LCC dataset test images"):
    feature_vec_LCC_test.append(feature_LBP(img))

# Convert lists to numpy arrays
feature_vector_LCC_train = np.array(feature_vec_LCC_train)
feature_vector_LCC_test = np.array(feature_vec_LCC_test)

Processing LCC dataset test images: 100%|██████████| 7635/7635 [00:18<00:00, 402.21it/s]


NUAA

In [9]:
from tqdm import tqdm

feature_vec_real = []
feature_vec_spoof = []

image_real = img_real
image_spoof = img_spoof

# Loop over images in image_real with tqdm progress bar
for img in tqdm(image_real, desc="Processing real images"):
    feature_vec_real.append(feature_LBP(img))

# Loop over images in image_spoof with tqdm progress bar
for img in tqdm(image_spoof, desc="Processing spoof images"):
    feature_vec_spoof.append(feature_LBP(img))

# Convert lists to numpy arrays
feature_vectors_real = np.array(feature_vec_real)
feature_vectors_spoof = np.array(feature_vec_spoof)

Processing spoof images: 100%|██████████| 7509/7509 [00:18<00:00, 403.32it/s]


In [10]:
from sklearn.model_selection import train_test_split

# Concatenating the Dataset

data_set = np.concatenate((feature_vectors_real, feature_vectors_spoof), axis=0)
data_set_label = np.concatenate((label_r, label_s),axis=0)

# Shuffling teh dataset
shuffle_index = np.random.permutation(len(data_set_label))

dataset, d_labels = data_set[shuffle_index,:], data_set_label[shuffle_index]

# Split the data into training and testing sets
nuaa_train, nuaa_test, nuaa_label_train, nuaa_label_test = train_test_split(dataset, d_labels, test_size=0.2, random_state=42)

In [11]:
# Arranging the dataset

NUAA = {"train_data":nuaa_train, "test_data":nuaa_test, "train_label":nuaa_label_train, "test_label":nuaa_label_test}
LCC = {"train_data":feature_vector_LCC_train,"test_data":feature_vector_LCC_test,"train_label":LCC_label_train,"test_label":LCC_label_test}
REPLAY = {"train_data":feature_vector_replay_train,"test_data":feature_vector_replay_test,"train_label":replay_label_train,"test_label":replay_label_test}
data_set = {"NUAA":NUAA,"LCC":LCC,"REPLAY":REPLAY}


In [12]:
# Save each numpy array in each dictionary as .npy files
for dataset_name, dataset_content in data_set.items():
    for data_name, data_array in dataset_content.items():
        np.save(f"LBP_{dataset_name}_{data_name}.npy", data_array)